## Import Data

In [1]:
import numpy as np
import imdb 
import pandas as pd
#from error_generator import Explicit_Missing_Value
#from error_generator import Implicit_Missing_Value
#from error_generator import White_Noise
#from error_generator import Gaussian_Noise
from error_generator import Random_Active_Domain
from error_generator import Similar_Based_Active_Domain
from error_generator import Typo_Keyboard
from error_generator import Typo_Butterfingers
from error_generator import Word2vec_Nearest_Neighbor
from error_generator import Value_Selector
from error_generator import List_selected
from error_generator import Read_Write
from error_generator import Error_Generator
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LassoCV
from yellowbrick.regressor import AlphaSelection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

anime_data = pd.read_csv("../new_anime_data1.csv", index_col=0)

anime_data['episodes'] = anime_data['episodes'].replace('Unknown', np.nan)


c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Cleaning

In [2]:
l1 = []
l2 = []
l3 = []


anime_data['genre'].fillna('',  inplace=True)
anime_data['overview'].fillna('',  inplace=True)
anime_data['type'].fillna('',  inplace=True)
for index, row in anime_data.iterrows():
    item = row['genre']
    if(pd.isnull(item)):
            item =""
    else:
        if isinstance(item, (list, tuple)):
            item = ','.join(item)
        else:
            item = item.replace(" ","")
            item = item.replace("[","")
            item = item.replace("]","")
            item = item.replace("'","")
    l1.append(item) 
      

for index, row in anime_data.iterrows():
    item = row['overview']
    if(pd.isnull(row['overview'])):
            item = ""
    else:
       # for item in anime_data['overview']:
            if isinstance(item, (list, tuple)):
                item = ','.join(item)
            else:
                item = item.replace("[","")
                item = item.replace("]","")
    l2.append(item) 

for index, row in anime_data.iterrows():
    item = row['type']
    if(pd.isnull(row['type'])):
        item = np.nan
    else:    
        if "movie" in item:
            item = "movie"
        else:
            item = "tv series"
    l3.append(item)   
    
    
anime_data['genre'] = l1   
anime_data['overview'] = l2
anime_data['type'] = l3

#drop dublicate
anime_data.drop_duplicates(inplace = True)



## Sampling

In [3]:
anime_train, anime_test = train_test_split(anime_data, test_size=0.2)
anime_test.to_csv("testDataset.csv", index = False,
                  columns = ['anime_id', 'name','genre','type', 'episodes','rating', 'members', 'overview'])

## Generating Errors

In [4]:
def error_generater(dataset, method, n, ignored_columns):

    myselector=List_selected()

    mygen=Error_Generator()

    new_dataset=mygen.error_generator(method_gen=method,selector=myselector,percentage=n,dataset=dataset,mute_column = ignored_columns)
    
    return new_dataset

In [5]:
# generate error in the training set
new_dataset = error_generater(anime_train.values.tolist(), Random_Active_Domain(),50, ignored_columns = [0,1,4,5,6])
Read_Write.write_csv_dataset("./{}.csv".format(Random_Active_Domain().name), new_dataset)
anime_train = pd.read_csv("./{}.csv".format(Random_Active_Domain().name), 
                            names = ['anime_id', 'name','genre','type', 'episodes','rating', 'members', 'overview'])
anime_train.to_csv("dirtyTrainDataset.csv", index = False,
                  columns = ['anime_id', 'name','genre','type', 'episodes','rating', 'members', 'overview'])

---------Change according to random_active_domain method ---------------

row: 767 col: 7 : ''Kanae and Sosuke are high school students who live in the medium-sized Japanese city of Hamamatsu. One day, "Classicaloid" versions of Beethoven and Mozart appear at Otowa Mansion, a large estate Kanae inherited from her grandmother. When the suspicious-looking ClassicaLoids play music they call "Musik," strange things start to happen. Eventually, more ClassicaLoids appear, such as Liszt, Bach, Chopin, and Schubert.'' changed to ''  
row: 442 col: 7 : ''Attending the private school Suzaku High Ryu Yamada is a student completely bored with his surroundings. One day while following the honor student Urara Shiraishi up the stairs falls down on her waking up to have switched bodies by accidentally kissing. Finding out their ability to to swap bodies Toranosuke Miyamura, the student councils president discovers their abilities to go into one and others bodies offering them the abandoned supernatura

## Transformation

In [6]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
def get_words(x):
    bagofwords=[]
    for i in x:
        if i[1]=='NN':
            bagofwords.append(i[0])
        elif i[1]=='NNS':
            bagofwords.append(i[0])
        elif i[1]=='NNP':
            bagofwords.append(i[0])
        elif i[1]=='NNPS':
            bagofwords.append(i[0])
        elif i[1]=='JJ':
            bagofwords.append(i[0])
        elif i[1]=='JJR':
            bagofwords.append(i[0])
        elif i[1]=='JJS':
            bagofwords.append(i[0])
        elif i[1]=='RB':
            bagofwords.append(i[0])
        elif i[1]=='RBR':
            bagofwords.append(i[0])
        elif i[1]=='RBS':
            bagofwords.append(i[0])
    return bagofwords

def clean_words(x):
    b=nltk.pos_tag(nltk.word_tokenize(x))
    result=get_words(b)
    return result

In [7]:
'''
def feature_transformation(dataset):
    #Hot_Encoding
    df = dataset.genre.str.get_dummies(',')
    
    type_lb = LabelBinarizer()
    X = type_lb.fit_transform(dataset.type.values)

    dfOneHot = pd.DataFrame(X, columns = ["movie/TVseries" for i in range(X.shape[1])])
    dataset = pd.concat([dataset, dfOneHot], axis=1)
    dataset = pd.concat([dataset, df], axis=1)

    dataset['movie/TVseries'].fillna(0, inplace=True)
    
    #Bag of Words
    summary_doc = dataset['overview'].fillna("").map(clean_words)
    summary_doc =summary_doc.apply(','.join)
 
    vectorizer = TfidfVectorizer()
    overview_feature = vectorizer.fit_transform(summary_doc).toarray()

    df = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])
    dataset = pd.concat([dataset, df], axis=1)
    
    dataset = dataset.drop(columns=['Unnamed: 0', 'anime_id', 'name', 'genre', 'overview', 'type'])
    
    #drop NaN values
    dataset = dataset.dropna()

    return dataset
'''

'\ndef feature_transformation(dataset):\n    #Hot_Encoding\n    df = dataset.genre.str.get_dummies(\',\')\n    \n    type_lb = LabelBinarizer()\n    X = type_lb.fit_transform(dataset.type.values)\n\n    dfOneHot = pd.DataFrame(X, columns = ["movie/TVseries" for i in range(X.shape[1])])\n    dataset = pd.concat([dataset, dfOneHot], axis=1)\n    dataset = pd.concat([dataset, df], axis=1)\n\n    dataset[\'movie/TVseries\'].fillna(0, inplace=True)\n    \n    #Bag of Words\n    summary_doc = dataset[\'overview\'].fillna("").map(clean_words)\n    summary_doc =summary_doc.apply(\',\'.join)\n \n    vectorizer = TfidfVectorizer()\n    overview_feature = vectorizer.fit_transform(summary_doc).toarray()\n\n    df = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])\n    dataset = pd.concat([dataset, df], axis=1)\n    \n    dataset = dataset.drop(columns=[\'Unnamed: 0\', \'anime_id\', \'name\', \'genre\', \'overview\', \'type\'])\n    \n    #dr

In [10]:
def get_dummies(train_data_set, test_data_set):
    train_dummies = train_data_set.genre.str.get_dummies(',')
    test_dummies = test_data_set.genre.str.get_dummies(',')
    
    print("Train Dummies",train_dummies.shape)    
    print("Test Dummies",test_dummies.shape)
    
    #### ALİGN
    train_dummies, test_dummies = train_dummies.align(test_dummies, axis=1, join='left')
    test_dummies.fillna(0, inplace=True)
    
    type_lb = LabelBinarizer()
    fitted_type_lb = type_lb.fit(train_data_set.type.values)
    X_train = fitted_type_lb.transform(train_data_set.type.values)
    X_test = fitted_type_lb.transform(test_data_set.type.values)
    
    dfOneHot_train = pd.DataFrame(X_train, columns = ["movie/TVseries" for i in range(X_train.shape[1])])
    dfOneHot_test = pd.DataFrame(X_test, columns = ["movie/TVseries" for i in range(X_test.shape[1])])
    
    train_data_set = pd.concat([train_data_set, dfOneHot_train], axis=1)
    train_data_set = pd.concat([train_data_set, train_dummies], axis=1)

    test_data_set = pd.concat([test_data_set, dfOneHot_test], axis=1)
    test_data_set = pd.concat([test_data_set, test_dummies], axis=1)
    
    test_data_set['movie/TVseries'].fillna(0, inplace=True)
    train_data_set['movie/TVseries'].fillna(0, inplace=True)

    return([train_data_set, test_data_set])

def feature_transformation(train_data_set, test_data_set):
    
    dummieset = get_dummies(train_data_set, test_data_set)
    train_data_set = dummieset[0]
    test_data_set = dummieset[1]
        
    #Bag of Words
    summary_doc_train = train_data_set['overview'].fillna("").map(clean_words)
    summary_doc_train =summary_doc_train.apply(','.join)
    
    summary_doc_test = test_data_set['overview'].fillna("").map(clean_words)
    summary_doc_test =summary_doc_test.apply(','.join)
 
    vectorizer = TfidfVectorizer()
    fitted_vectorizer = vectorizer.fit(summary_doc_train)
    overview_feature_train = fitted_vectorizer.transform(summary_doc_train).toarray()
    overview_feature_test = fitted_vectorizer.transform(summary_doc_test).toarray()

    df_train = pd.DataFrame(overview_feature_train, columns = ["word"+ str(int(i)) for i in range(overview_feature_train.shape[1])])
    train_data_set = pd.concat([train_data_set, df_train], axis=1)
    
    df_test = pd.DataFrame(overview_feature_test, columns = ["word"+ str(int(i)) for i in range(overview_feature_test.shape[1])])
    test_data_set = pd.concat([test_data_set, df_test], axis=1)
    
    train_data_set = train_data_set.drop(columns=['anime_id', 'name', 'genre', 'overview', 'type'])
    test_data_set = test_data_set.drop(columns=['anime_id', 'name', 'genre', 'overview', 'type'])
    
    #drop NaN values
    train_data_set.dropna(inplace=True)
    test_data_set.dropna(inplace=True)
    
    #train_data_set.fillna(0, inplace=True)
    #test_data_set.fillna(0, inplace=True)
    
    return ([train_data_set, test_data_set])



In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

'''
anime_train = feature_transformation(anime_train)
anime_test = feature_transformation(anime_test)
'''
transformed_features = feature_transformation(anime_train, anime_test)

anime_train = transformed_features[0]
anime_test = transformed_features[1]

Train Dummies (1600, 57)
Test Dummies (400, 53)


In [12]:
'''
anime_train_columns = list(anime_train)
anime_test_columns = list(anime_test)

for col_name in anime_train_columns:
    if col_name not in anime_test_columns:
        anime_test[str(col_name)] = 0
        
for col_name in anime_test_columns:
    if col_name not in anime_train_columns:
        anime_test.drop([str(col_name)], axis=1, inplace=True)
        
'''

'\nanime_train_columns = list(anime_train)\nanime_test_columns = list(anime_test)\n\nfor col_name in anime_train_columns:\n    if col_name not in anime_test_columns:\n        anime_test[str(col_name)] = 0\n        \nfor col_name in anime_test_columns:\n    if col_name not in anime_train_columns:\n        anime_test.drop([str(col_name)], axis=1, inplace=True)\n        \n'

In [13]:
#anime_train = anime_train.dropna()
print(anime_train.shape)
print(anime_test.shape)

(1544, 6329)
(26, 6329)


In [14]:
anime_y_train = anime_train['rating']
anime_X_train = anime_train.drop(columns=['rating'])

anime_y_test = anime_test['rating']
anime_X_test = anime_test.drop(columns=['rating'])


## Feature Selection

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler

selector = SelectKBest(score_func=f_regression,k=700)#anime_X_test.shape[1]-1)
features = selector.fit(anime_X_train, anime_y_train)

# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)

print(anime_X_train.shape)
print(anime_X_test.shape)
anime_X_train = features.transform(anime_X_train)
anime_X_test = features.transform(anime_X_test)
print(anime_X_train.shape)
print(anime_X_test.shape)

scaler = StandardScaler() 

# Apply transform to both the training set and the test set.
anime_X_train = scaler.fit_transform(anime_X_train)  
anime_X_test = scaler.transform(anime_X_test) 

c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


(1544, 6328)
(26, 6328)
(1544, 700)
(26, 700)


c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Training - Testing - Evaluation

In [16]:
def training(model, dataset, label):
    clf = model
    clf.fit(dataset, label)
    return clf

def testing_evaluation(model, testset):
    # Make predictions using the testing set
    anime_y_pred = model.predict(testset)

    # The mean absolute error
    print("Mean absolute error: %.2f" % np.sqrt(mean_absolute_error(anime_y_test, anime_y_pred)))

    # The mean squared error
    print("Mean squared error: %.2f" % np.sqrt(mean_squared_error(anime_y_test, anime_y_pred)))

    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(anime_y_test, anime_y_pred))

### Linear Regression

In [17]:
clf = training(model = linear_model.LinearRegression(), dataset = anime_X_train, label= anime_y_train)
testing_evaluation(clf, anime_X_test)

Mean absolute error: 2330739.23
Mean squared error: 21908396834020.90
Variance score: -7432703936705687571958595584.00


### Lesso Regression

In [18]:
clf = training(model = linear_model.Lasso(alpha=0.1), dataset = anime_X_train, label= anime_y_train)
testing_evaluation(clf, anime_X_test)

Mean absolute error: 1.13
Mean squared error: 1.36
Variance score: -27.79


### LassoCV

In [19]:
'''
# Create a list of alphas to cross-validate against
alphas = np.logspace(-10, 1, 100)

# Instantiate the linear model and visualizer
model = LassoCV(alphas=alphas, cv = 5)
visualizer = AlphaSelection(model)

visualizer.fit(anime_X_train, anime_y_train)
g = visualizer.poof()
'''


'\n# Create a list of alphas to cross-validate against\nalphas = np.logspace(-10, 1, 100)\n\n# Instantiate the linear model and visualizer\nmodel = LassoCV(alphas=alphas, cv = 5)\nvisualizer = AlphaSelection(model)\n\nvisualizer.fit(anime_X_train, anime_y_train)\ng = visualizer.poof()\n'

### Cross Validation

In [20]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
scores = cross_val_score(clf, anime_X_train, anime_y_train, scoring="neg_mean_squared_error", cv=5) 
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)

Scores: [0.882 0.942 0.966 0.98  0.935]
Mean: 0.9409869803229297
Standard deviation: 0.033549690717947195


### Grid Search For Hyper Parameter Selection

In [21]:
def checkHP(model, folds, dataset, label):
    parameters = {
                   "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                  }

    gd_sr = GridSearchCV(estimator=model,  
                         param_grid=parameters,
                         scoring="neg_mean_squared_error",
                         cv=folds)

    gd_sr.fit(dataset, label)  
    
    best_parameters = gd_sr.best_params_  
    print("best parameters are: ", best_parameters)

    best_result = gd_sr.best_score_  
    print("The mean squared Error is: %.2f" % -best_result) 
    
checkHP(clf, 5, anime_X_train, anime_y_train) 

c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\master\appdata\local\programs\python\python36-32\lib\si

best parameters are:  {'alpha': 0.01}
The mean squared Error is: 0.88


### Re-Training

In [22]:
clf = training(model = linear_model.Lasso(alpha=0.01), dataset = anime_X_train, label= anime_y_train)
testing_evaluation(clf, anime_X_test)

Mean absolute error: 1.12
Mean squared error: 1.49
Variance score: -33.29
